In [ ]:
import pandas as pd
import numpy as np
import igraph as ig

# Import the city data.
raw = []
with open('input.txt','r') as f:
    for line in f:
        if line.strip()[0] != "#":
            raw.append(line.split())
            
n_cities = int(raw[0][0])
reliability_1d = np.asarray(raw[1], dtype='double')
cost_1d = np.asarray(raw[2], dtype='int')

# Convert the 1D matrices to 2D.
def convert_to_2d(size, arr1d):
    new_array = np.zeros((size, size))
    k = 0
    for i in range(0, size):
        for j in range(i + 1, size):
            new_array[i][j] = arr1d[k]
            k = k + 1
    return new_array

reliability = convert_to_2d(n_cities, reliability_1d)
cost = convert_to_2d(n_cities, cost_1d)

# Print matrices and extracted information.
print("Number of cities: " + str(n_cities))
print("Reliability matrix:")
print(reliability)
print("Cost matrix:")
print(cost)

# Create reliability graph.
graph = ig.Graph.Weighted_Adjacency(reliability).as_undirected()
graph.es["reliability"] = reliability_1d
graph.es["cost"] = cost_1d
graph.es["curved"] = [False for i in reliability_1d]
graph.vs["label"] = range(n_cities)

In [ ]:
# Create reliability graph.
# Create list of edges
def find_edges(size, arr2d):
    new_array = []
    k = 0
    for i in range(0, size):
        for j in range(i + 1, size):
            if (arr2d[i][j] != 0):
                new_array.append((i,j))
    return new_array

graph = ig.Graph()
graph.add_vertices(n_cities)
graph.add_edges(find_edges(n_cities,reliability))
graph.es['reliability'] = reliability_1d
graph.es["cost"] = cost_1d
graph.es["curved"] = [False for i in reliability_1d]
graph.vs["label"] = list(range(n_cities))

In [ ]:
# Plot reliability graph.
ig.plot(graph, edge_label=[str(r) for r in graph.es["reliability"]], vertex_label=[str(r) for r in graph.vs["label"]])

In [ ]:
# Plot cost graph.
ig.plot(graph, edge_label=[str(c) for c in graph.es["cost"]])

In [ ]:
# Find the MST (minimum spanning tree) using Kruskal's algorithm. 1 - r = u, so minimize unreliability.
mst_graph = graph.spanning_tree(weights=[1 - r for r in graph.es["reliability"]])

In [ ]:
# Plot the MST graph (optimized for minimum unreliability),
ig.plot(mst_graph, edge_label=[str(r) for r in mst_graph.es["reliability"]])

In [ ]:
# Compute cost and reliability for the MST.
import functools
import operator

mst_cost = sum(mst_graph.es["cost"])
# Since it's a minimum spanning tree, if any of the edges has failed, the entire system has failed.
mst_reliability = functools.reduce(operator.mul, mst_graph.es["reliability"])

print(f"The reliability of the MST is {mst_reliability:.3f} and its cost is {mst_cost}.")

In [ ]:
# remove edge from graph and return new graph
def remove_edge(graph, edge):
    new_graph = graph.copy()
    new_graph.delete_edges([edge])
    if (new_graph.degree(edge[0]) == 0):
        dead_vertice = edge[0]
        new_graph.delete_vertices(dead_vertice)
    if (new_graph.degree(edge[1]) == 0):
        dead_vertice = edge[1]
        new_graph.delete_vertices(dead_vertice)
    return new_graph

# contract vertices in graph and return new graph
def contract_vertices(graph, edge):
    new_graph = graph.copy()
    new_labels = new_graph.vs["label"]
    vertices = new_labels.copy()
    dead_vertice = min(edge)
    surviving_vertice = max(edge)
    index_dead = vertices.index(dead_vertice)
    vertices[index_dead] = surviving_vertice
    new_graph.contract_vertices(vertices)
    new_graph.delete_edges([(surviving_vertice,surviving_vertice)])
    new_graph.delete_vertices(dead_vertice)
    new_labels.remove(dead_vertice)
    new_graph.vs["label"] = new_labels
    return new_graph
    
# find reliability by using edge decomposition recursively
def find_graphR(graph):
    vertice_count = graph.vcount();
    edge_count = graph.ecount();
    # when graph has one vertice with self-loops
    if (vertice_count == 1 and edge_count > 0):
        return 1
    # when graph is a tree (no loops)
    elif (graph.is_tree):
        r = functools.reduce(operator.mul, graph.es["reliability"])
        return r
    # when graph is connected and has loops
    elif (graph.is_connected()):
        edges = graph.get_edgelist()
        reliabilities = graph.es["reliability"]
        min_r = min(reliabilities)
        index = reliabilities.index(min_r)
        edge_r = edges[index]
        graph_without_edge = remove_edge(graph, edge_r)
        graph_with_edge = contract_vertices(graph, edge_r)
        return (1-min_r)*find_graphR(graph_without_edge) + min_r*find_graphR(graph_with_edge)
    # when graph is disconnected
    else:
        print("algorithm failed because we deleted an edge that connects two different parts of the graph")
        return 0

In [ ]:
original_edge_list = graph.get_edgelist()
original_reliabilty_list = graph.es["reliability"]
mst_edge_list = mst_graph.get_edgelist()
mst_reliabilty_list = mst_graph.es["reliability"]
print(original_edge_list)
print(original_reliabilty_list)
print(mst_edge_list)
print(mst_reliabilty_list)

remaining_edge_list = original_edge_list.copy()
for x in mst_edge_list:
    remaining_edge_list.remove(x)
print(remaining_edge_list)

remaining_r_list = original_reliabilty_list.copy()
for x in mst_reliabilty_list:
    remaining_r_list.remove(x)
print(remaining_r_list)

# find new graph given target reliability
def get_target_graph_and_r(r_goal, mst, edge_list_a, r_list_a):
    new_reliability = 0
    available_edge_list = edge_list_a.copy()
    available_r_list = r_list_a.copy()
    new_graph = mst.copy()
    new_graph_r_list = new_graph.es["reliability"].copy()
    while len(available_r_list) > 0:
        max_r = max(available_r_list)
        index = available_r_list.index(max_r)
        new_edge = available_edge_list[index]
        new_graph_r_list.append(max_r)
        new_graph.add_edges([new_edge])
        new_graph.es["reliability"] = new_graph_r_list
        available_edge_list.remove(new_edge)
        available_r_list.remove(max_r)
        new_reliability = find_graphR(new_graph)
        if new_reliability >= r_goal:
            return (new_reliability)
    return new_reliability

In [ ]:
final_r = get_target_graph_and_r(0.9, mst_graph, remaining_edge_list, remaining_r_list)
print(final_r)